In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import pygrib
import datetime
import numexpr as ne

['vis', 'gust', 'gh', 't', 'r', 'tcc', 'u', 'v', 'orog', '2t', '2r', '10u', '10v', 'prate', 'tp', 'lftx', 'cape']


#### TO DO : DEWPOINT 2rh ganti jadi RH, wind belom hitung

In [46]:
grbs = pygrib.open('/home/opn/script/siam-netcdf-generator/coba/tes/envyosa/gfsdowntemp/20231127/t12/t12.f001')
grbs1 = pygrib.open('/home/opn/script/siam-netcdf-generator/coba/tes/envyosa/gfsdowntemp/20231127/t12/t12.f003')

In [47]:
gsel = grbs.select(shortName ='2r')
gsel1 = grbs1.select(shortName ='2r')

In [48]:
gsel1[0].values

array([[77.3, 77.9, 75.5, ..., 73. , 76.8, 79.6],
       [77. , 77.6, 71. , ..., 74.7, 80. , 79.9],
       [74.4, 73.2, 73.5, ..., 78. , 80.6, 80.6],
       ...,
       [79. , 85.5, 94.3, ..., 71.2, 72.1, 73.1],
       [80.3, 80.8, 81. , ..., 75.9, 78. , 78.4],
       [79.9, 80.5, 83.2, ..., 78.5, 78.6, 79. ]])

In [42]:
gsel[0].values

array([[75.1, 75.1, 77.5, ..., 72.4, 72.4, 75.3],
       [72.8, 73.3, 74.6, ..., 73.7, 74. , 79. ],
       [72.1, 72.5, 74.6, ..., 74.6, 77.3, 80.4],
       ...,
       [79.8, 84.2, 92.7, ..., 72.1, 73.2, 73.8],
       [78.3, 79.6, 79.8, ..., 76.1, 76.6, 77.7],
       [77.7, 79.1, 79.2, ..., 77.7, 77.9, 79.7]])

In [4]:
def mstoknot(var):
    return ne.evaluate('var*1.9438445')

def KtodegC(var):
    return ne.evaluate('var-273.15')

def patombar(var):
    return ne.evaluate('var*0.01')
def remask(var,varmask):
    import numpy.ma as ma
    masktemp = varmask.mask
    var = ma.masked_array(var,mask=masktemp)
    return var
param2d = [
    ['2r', 'rh2', None, '%',None],                  # Relative humidity at 2m
    ['2t', 't2', KtodegC, 'degC',None],             # Temperature at 2m
    ['10u', 'u10', None, 'm s**-1',None],           # U-component of wind at 10m
    ['10v', 'v10', None, 'm s**-1',None],           # V-component of wind at 10m
    ['vis', 'vis', None, 'm',None],                 # Visibility
    ['gust', 'gust', None, 'm s**-1',None],         # Wind gust
    ['prate', 'prate', None, 'kg m**-2 s**-1','avg'],# Precipitation rate
    ['tp', 'rr', None, 'mm',None],                  # Total precipitation
#     ['acpcp', 'acpcp', None, 'kg m**-2'],      # Accumulated convective precipitation
    ['lftx', 'lftx', None, 'K',None],               # Surface lifted index
    ['cape', 'cape', None, 'J kg**-1',None],        # Convective Available Potential Energy
    ['tcc', 'tcc_atmospheric', None,'%','avg']
]

param3d = [
    ['u', 'u', None, 'm s**-1'],      # U-component of wind
    ['v', 'v', None, 'm s**-1'],      # V-component of wind
    ['gh', 'gh', None, 'gpm'],        # Geopotential Height
    ['t', 't', KtodegC, 'C'],            # Temperature
    ['r', 'r', None, '%'],            # Relative Humidity
    ['tcc', 'tcc_isobaric', None, '%'],        # Total Cloud Cover
]

# yang ada datanya tapi ga masuk sini orog

In [5]:
gsel = grbs.select(shortName = param2d[1][0]) #rh
g = gsel[0]
lat, lon = g.latlons()
lat = lat[:,0]
lon = lon[0,:]
dtni = g.analDate + datetime.timedelta(hours = g['endStep'])
tunits = 'minutes since %s' % dtni.strftime('%Y-%m-%d %H:%M')

In [6]:
levels = []
for allvar in param3d:
    gselz = grbs.select(shortName = allvar[0])
    for lev in gselz:
        levels.append(float(lev.level))
levels = np.unique(levels)  

In [7]:
timestep = pd.date_range(start=dtni, periods=240, freq='H')

In [8]:
ds = xr.Dataset(coords=dict(
        lon=(["lon"], lon),
        lat=(["lat"], lat),
        level=(["level"], levels),
        time=(["time"],timestep)))

In [9]:
for allvar in param2d:
    dummy = np.empty([len(timestep),len(lat),len(lon)])
    ds[allvar[1]] = (('time','lat','lon'),dummy)
for allvar in param3d:
    dummy = np.empty([len(timestep),len(levels), len(lat), len(lon)])
    ds[allvar[1]] = (('time','level','lat','lon'),dummy)

In [10]:
for allvar in param2d:
    if allvar[4] != None:
        gsel = grbs.select(shortName = allvar[0],stepType=allvar[4])
    else:
        gsel = grbs.select(shortName = allvar[0]) #rh
    g = gsel[0]
    var = g.values
    converter = allvar[2]
    if converter:
        if type(var) is np.ma.core.MaskedArray:
            vartemp = var
            var = converter(var)
            var = remask(var,vartemp)
        else:
            var = converter(var)
    ds[allvar[1]][0,:,:] = var #nanti indexingnya benerin buat for loop

In [12]:
for allvar in param3d:
    data = np.empty([len(levels), len(lat), len(lon)])

    g_sel = grbs.select(shortName = allvar[0],
                        typeOfLevel = 'isobaricInhPa')
    converter = allvar[2]
    
    i=0
    if converter:      
        for gselsel in g_sel:
            data = gselsel.values
            if type(data) is np.ma.core.MaskedArray:
                vartemp = data
                data = converter(data)
                data = remask(data,vartemp)
            else:
                data = converter(data)
            ds[allvar[1]][0,i,:,:]=data
            i+=1
    else : 
        for gselsel in g_sel:
            data = gselsel.values
            ds[allvar[1]][0,i,:,:]=data
            i+=1


In [14]:
grbs.seek(0)
gsel1 = grbs.select(shortName = 'tcc',typeOfLevel = 'isobaricInhPa') #rh
gsel1[1].values == ds['tcc_isobaric'][0,1,:,:].values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [77]:
grbs.seek(0)
for g in grbs:
    print(f'{g.typeOfLevel}')

surface
surface
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isobaricInhPa
isob

In [18]:
chunked = ds.chunk('auto')

In [27]:
chunked

<xarray.Dataset>
Dimensions:          (lat: 69, level: 16, lon: 185, time: 240)
Coordinates:
  * lon              (lon) float64 95.0 95.25 95.5 95.75 ... 140.5 140.8 141.0
  * lat              (lat) float64 -11.0 -10.75 -10.5 -10.25 ... 5.5 5.75 6.0
  * level            (level) float64 0.0 100.0 150.0 200.0 ... 925.0 950.0 1e+03
  * time             (time) datetime64[ns] 2023-11-22T07:00:00 ... 2023-12-02...
Data variables:
    rh2              (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    t2               (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    u10              (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    v10              (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    vis              (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    gust             (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    prate            (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    rr               (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    lftx             (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    cape             (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    tcc_atmospheric  (time, lat, lon) float64 dask.array<chunksize=(240, 69, 185), meta=np.ndarray>
    u                (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>
    v                (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>
    gh               (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>
    t                (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>
    r                (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>
    tcc_isobaric     (time, level, lat, lon) float64 dask.array<chunksize=(120, 16, 69, 123), meta=np.ndarray>

In [45]:
chunk2d = [3,6,12]
chunk3d = [3,3,6,12]
encode = {"rh2": {'zlib' : True, "complevel":5, "chunksizes":chunk2d },
          "t2": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "u10": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "v10": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "vis": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "gust": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "prate": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "rr": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "lftx": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "cape": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "tcc_atmospheric": {'zlib' : True, "complevel":5,"chunksizes":chunk2d},
          "u": {'zlib' : True, "complevel":5,"chunksizes":chunk3d},
          "v": {'zlib' : True, "complevel":5,"chunksizes":chunk3d},
          "gh": {'zlib' : True, "complevel":5,"chunksizes":chunk3d},
          "t": {'zlib' : True, "complevel":5,"chunksizes":chunk3d},
          "r": {'zlib' : True, "complevel":5,"chunksizes":chunk3d},
          "tcc_isobaric": {'zlib' : True, "complevel":5,"chunksizes":chunk3d}
          } 

In [46]:
ds.to_netcdf("gfs/outncteschunked.nc",encoding = encode)

In [47]:
dscoba = xr.open_dataset("gfs/outncteschunked.nc")

In [48]:
dscoba

<xarray.Dataset>
Dimensions:          (lat: 69, level: 16, lon: 185, time: 240)
Coordinates:
  * lon              (lon) float64 95.0 95.25 95.5 95.75 ... 140.5 140.8 141.0
  * lat              (lat) float64 -11.0 -10.75 -10.5 -10.25 ... 5.5 5.75 6.0
  * level            (level) float64 0.0 100.0 150.0 200.0 ... 925.0 950.0 1e+03
  * time             (time) datetime64[ns] 2023-11-22T07:00:00 ... 2023-12-02...
Data variables:
    rh2              (time, lat, lon) float64 ...
    t2               (time, lat, lon) float64 ...
    u10              (time, lat, lon) float64 ...
    v10              (time, lat, lon) float64 ...
    vis              (time, lat, lon) float64 ...
    gust             (time, lat, lon) float64 ...
    prate            (time, lat, lon) float64 ...
    rr               (time, lat, lon) float64 ...
    lftx             (time, lat, lon) float64 ...
    cape             (time, lat, lon) float64 ...
    tcc_atmospheric  (time, lat, lon) float64 ...
    u                (time, level, lat, lon) float64 ...
    v                (time, level, lat, lon) float64 ...
    gh               (time, level, lat, lon) float64 ...
    t                (time, level, lat, lon) float64 ...
    r                (time, level, lat, lon) float64 ...
    tcc_isobaric     (time, level, lat, lon) float64 ...

In [37]:
ds['rh2']

<xarray.DataArray 'rh2' (time: 240, lat: 69, lon: 185)>
array([[[76.9, 76.7, 78.4, ..., 81.5, 79.5, 79.2],
        [76. , 75.7, 76.3, ..., 80.4, 79.7, 81.2],
        [74.2, 73.7, 74.1, ..., 81.3, 81.5, 83. ],
        ...,
        [72.4, 73.9, 77.4, ..., 75.3, 75.4, 75.2],
        [70.9, 72.2, 73.2, ..., 75.5, 75.5, 75.3],
        [70.2, 71.5, 71.9, ..., 75.4, 75.6, 75.7]],

       [[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        ...,
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]],

       [[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        ...,
...
        ...,
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]],

       [[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        ...,
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]],

       [[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        ...,
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]]])
Coordinates:
  * lon      (lon) float64 95.0 95.25 95.5 95.75 ... 140.2 140.5 140.8 141.0
  * lat      (lat) float64 -11.0 -10.75 -10.5 -10.25 -10.0 ... 5.25 5.5 5.75 6.0
  * time     (time) datetime64[ns] 2023-11-22T07:00:00 ... 2023-12-02T06:00:00